### Импорты

In [18]:
import joblib
import cv2
import kagglehub
import numpy as np
import os
import warnings

from collections import defaultdict
from sklearn.model_selection import train_test_split
from src.image_classification.features import (
    AmountOfYellow,
    AmountOfSilver,
    AmountOfParallelLines,
    AmountOfCylinders,
    AmountOfReflections,
    AmountOfTransparency,
    AmountOfTextureSmoothness,
    AmountOfTextureShininess,
    AmountOfSurfaceAnisotropy,
    AmountOfAspectRatio,
    AmountOfWhiteness,
    AmountOfLineCurvature,
)

### Отключение ворнингов

In [19]:
warnings.filterwarnings("ignore")

Загрузка классификатора и разбиение изображений

In [20]:
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
classes = os.listdir(os.path.join(path, "garbage-dataset"))


def get_features(img):
    return (AmountOfYellow()(img),
            AmountOfSilver()(img),
            AmountOfParallelLines()(img),
            AmountOfCylinders()(img),
            AmountOfReflections()(img),
            AmountOfTransparency()(img),
            AmountOfTextureSmoothness()(img),
            AmountOfTextureShininess()(img),
            AmountOfSurfaceAnisotropy()(img),
            AmountOfAspectRatio()(img),
            AmountOfWhiteness()(img),
            AmountOfLineCurvature()(img),)

test_images = [] 
features = []
labels = []
for class_name in classes:
    class_path = os.path.join(path + "/garbage-dataset/", class_name)
    image_names = os.listdir(class_path)

    # Split the images into training and testing sets (80% train, 20% test)
    train_images, test_images_class = train_test_split(image_names, test_size=0.2, random_state=42)

    # Collect test images for manual testing
    for image_name in test_images_class:
        test_images.append(os.path.join(class_path, image_name))

classifier = joblib.load('random_forest_classifier.pkl')

def classify_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        features_list = get_features(image)
        feature = np.array([features_list])  
        prediction = classifier.predict(feature)
        return prediction[0]
    else:
        return None

### Тест классификатора на разных клаасах

In [22]:
def test_classify(test_images, target_class_name):
    predictions = defaultdict(int)
    total_count = 0
    correct_count = 0

    for test_image in test_images:
        if target_class_name in test_image:  
            total_count += 1  # Increment total count for the target class
            predicted_class = classify_image(test_image)  
            predictions[predicted_class] += 1 
            if predicted_class == target_class_name:
                correct_count += 1  # Increment correct count if prediction matches target class

    return dict(predictions), total_count, correct_count  

for class_name in classes:
    results, total_count, correct_count = test_classify(test_images, class_name)
    print(f"Predicted class counts for test images containing '{class_name}':")
    for predicted_class, count in results.items():
        print(f"Class: {predicted_class}, Count: {count}")
    print(f"Correctly guessed {correct_count} out of {total_count} for class '{class_name}'.")

Predicted class counts for test images containing 'trash':
Class: trash, Count: 87
Class: clothes, Count: 11
Class: plastic, Count: 12
Class: glass, Count: 45
Class: paper, Count: 4
Class: shoes, Count: 9
Class: metal, Count: 6
Class: battery, Count: 3
Class: cardboard, Count: 10
Class: biological, Count: 3
Correctly guessed 87 out of 190 for class 'trash'.
Predicted class counts for test images containing 'glass':
Class: glass, Count: 365
Class: metal, Count: 10
Class: trash, Count: 15
Class: plastic, Count: 37
Class: shoes, Count: 54
Class: clothes, Count: 75
Class: biological, Count: 18
Class: cardboard, Count: 8
Class: paper, Count: 14
Class: battery, Count: 17
Correctly guessed 365 out of 613 for class 'glass'.
Predicted class counts for test images containing 'battery':
Class: shoes, Count: 29
Class: glass, Count: 23
Class: battery, Count: 116
Class: biological, Count: 11
Class: trash, Count: 1
Class: clothes, Count: 4
Class: cardboard, Count: 3
Class: plastic, Count: 1
Class: me